In [10]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from dotenv import dotenv_values
from pprint import pprint

In [6]:
env_config = dotenv_values("../environment/.env")

In [8]:
es_client = Elasticsearch(f"http://{env_config['ELASTICSEARCH_HOST']}:{env_config['ELASTICSEARCH_PORT']}")

In [9]:
model = SentenceTransformer(env_config["SENTENCE_TRANSFORMER_MODEL"])

## Todos los resultados

In [17]:
params = {
    "index": "boe",
    "query": {"match_all": {}},
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint(res["hits"])

{'hits': [{'_id': 'desUtIsBir71ATlveUwl',
           '_index': 'boe',
           '_score': 1.0,
           '_source': {'doc_id': 'BOE-B-2023-32033',
                       'texto': 'REF.: MC-0019/2022 (INTEGRA AYE) VA'}},
          {'_id': 'dusUtIsBir71ATlveUz6',
           '_ignored': ['texto.keyword'],
           '_index': 'boe',
           '_score': 1.0,
           '_source': {'doc_id': 'BOE-B-2023-32033',
                       'texto': 'D. David Martín García (***4191**), en '
                                'representación de la Comunidad de Regantes '
                                'Finca Valijara (en formación), solicita de la '
                                'Confederación Hidrográfica del Duero una '
                                'modificación de características de concesión '
                                'de un aprovechamiento de aguas subterráneas, '
                                'otorgada a favor de D. Sebastián Martín '
                                'Martín y D

## Resultados semánticos con KNN

In [21]:
query_text = "cantidad de agua tope"
knn_params = {
    "field": "embedding",
    "query_vector": model.encode(query_text),
    "k": 2,
    "num_candidates": 5,
}
params = {
    "index": "boe",
    "knn": knn_params,
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint(res["hits"])

{'hits': [{'_id': 'fusUtIsBir71ATlve0xK',
           '_index': 'boe',
           '_score': 0.804345,
           '_source': {'doc_id': 'BOE-B-2023-32033',
                       'texto': '- Las aguas captadas se prevén tomar de la '
                                'Masa de agua subterránea: Terciario Detrítico '
                                'Bajo los Páramos (2º ciclo).'}},
          {'_id': 'dusUtIsBir71ATlveUz6',
           '_ignored': ['texto.keyword'],
           '_index': 'boe',
           '_score': 0.7206449,
           '_source': {'doc_id': 'BOE-B-2023-32033',
                       'texto': 'D. David Martín García (***4191**), en '
                                'representación de la Comunidad de Regantes '
                                'Finca Valijara (en formación), solicita de la '
                                'Confederación Hidrográfica del Duero una '
                                'modificación de características de concesión '
                                'de

## Combinado

In [23]:
query_text = "cantidad de agua tope"
query_params = {
    "lexical_filter": {
        "fields": "text",
        "query": "expediente",
    }
}
knn_params = {
    "field": "embedding",
    "query_vector": model.encode(query_text),
    "k": 2,
    "num_candidates": 5,
}
params = {
    "index": "boe",
    "query": query_params,
    "knn": knn_params,
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint(res["hits"])

BadRequestError: BadRequestError(400, 'parsing_exception', 'unknown query [lexical_filter]')